# Design of a Tension Member and End Connections

This design examples considers an HSS tension member, slotted to accept bolted plate connections.

The following data are given:

| Property | Value |
|----------|:-------:|
|  $T_f$   | 800 kN|
| Angles   | 300W  |
| Plates   | 300W |
| HSS      | 350W |
| W        | A992 |
| Bolts    | A325 3/4" |
| Holes    | 22 mm punched |
| Electrodes | E49xx |

In [1]:
# set the default precion for display of floating point numbers
%precision 3

# import useful utilities
from sst import SST
sst = SST()
from utils import show

In [2]:
phi = 0.9
phiu = 0.75
phib = 0.8
phibr = 0.8
phiw = 0.67

aFy = 300   # angles
aFu = 440
pFy = 300   # plates
pFu = 440
hFy = 350   # HSS
hFu = 450
wFy = 345   # W shapes
wFu = 450
bFu = 825   # bolt
Xu = 490    # weld
bd = 25.4 * 0.75   # 3/4" bolt
hd = 22 + 2       # in 22mm punched holes

In [3]:
Tf = 800     # Factored applied load

## 1: HSS Tension Member

Tension brace will be bolted into frame.  Net area fracture is often the governing case.  Reasonable
estimates are $A_{n} = 0.9 A_g$ and $A_ne = 0.85 A_n$.  Therefore, a first estimate of cross-sectional
area required is, without differentiating material strengths, is:

$$
\phi_u \times 0.85 \times 0.9 A_g \times Fu \ge T_f
$$

$$
A_g \ge \frac{T_f}{\phi_u 0.85 \times 0.9 Fu}
$$

In [4]:
Tf/(phi*0.85*0.9*450*1E-3)

2582.103

Therefore, look for a square HSS of at least this cross-sectional area:

>   _Gentle Reader:_ If you find the following 'python-pandas magic' hard to understand,
>   don't worry about it.  Its just a digital equivalent of searching through
>   the properties and dimensions tables in Part 6 of the CISC handbook of Steel Construction.

In [5]:
# load a table of all HSS sections, sorted by increasing mass
hss = sst.section_tables(['HS'],'Mass')

# search for those shapes whose area meets the limit and whose B and D dimensions are equal (square)
# Then extract the first 10 rows (shapes) of the search result and choose only three columns for display.
hss.query('A>2500 & B==D').head(10)[['A','T','Mass']]

,A,T,Mass
Dsg,,,
HS114x114x6.4,2640.0,6.35,20.7
HS152x152x4.8,2760.0,4.78,21.7
HS89x89x9.5,2790.0,9.53,21.9
HS102x102x8.0,2820.0,7.95,22.1
HS127x127x6.4,2960.0,6.35,23.2
HS114x114x8.0,3220.0,7.95,25.3
HS178x178x4.8,3250.0,4.78,25.5
HS102x102x9.5,3280.0,9.53,25.7
HS152x152x6.4,3610.0,6.35,28.3


Probably do not want the thinnest section (because of welding considerations). 
Also, a larger dimension may be easier to connect.
And leave a bit of allowance 
over the minimum.  Therefore:

### 1.1: Try: HSS 127x127x6.4

In a configuration similar to this:

![Photo](170-DSCN0768.JPG)

In [6]:
##angle=sst.section_tables(['L'],'Mass')
##angle.query("Avl!='*'&A>2500/4")[['A','T','Mass']].head(10)

In [7]:
##angle=sst.section_tables(['W'],'Mass')
##angle.query("Avl!='*'&A>2500")[['Dsg','A','T','Mass']].head(10)

## 2: Bolting Requirements

Again, because net area considerations usually govern we should start by selecting a bolt configuration (number of bolts and layout).  That will allow us to decide gusset plate thickness, which is important because the insert
plate welded into the HSS must be the same thickness, and that affects the strength of the HSS.

In [8]:
Ab = 3.14159*bd*bd/4   # bolt shear area
vr1 = 0.60*phib*1*2*Ab*bFu * 0.7 * 1E-3   # factored shear strength of 1 bolt, double shear, threads intercepted
Tf/vr1                 # number of bolts required

5.063

Therefore, use 6 bolts.  But should they be in a 2x3 or a 3x2 arrangement?

![Layout](bolt-layout.svg)

### 2.1: Try 2x3 bolt pattern
2 gauge lines, 3 transverse lines.  Use Gauge = 75mm, spacing = 75mm (minimum = 2.7d = 52mm).

From Table 6, minimum edge/end distance = 32mm (assuming sheared edges). Use edge and end distances of 35mm.

## 3: Gusset plate design
Mostly we want the thickness so that we can determine the thickness of the insert plate in the HSS.

As the gusset plate can be made as wide as necessary, design will likely be governed by block shear and/or bearing.

Calculate resistances of a 1mm thickness of plate, then determine the thickness required.

![Gusset Plate](gusset-plate.svg)

![Gusset Failure Paths](gusset-failure-paths.svg)

### 3.1: Path 1-2 (Block Shear) (1mm plate)

In [9]:
An = (75 - (hd/2)*2)*1
Ut = 1
An

51.000

In [10]:
Agv = (2*(2*75+35))*1
Agv

370

In [11]:
vr1 = phiu*(Ut*An*pFu + 0.6*Agv*(pFy+pFu)/2) * 1E-3
vr1

78.435

### 3.2: Path 1-3 (Block Shear)  (1mm plate)

In [12]:
An = (75+35-(hd/2)*3)*1
Ut = 0.6
An

74.000

In [13]:
Agv = (1*(2*75+35))*1
Agv

185

In [14]:
vr2 = phiu*(Ut*An*pFu + 0.6*Agv*(pFy+pFu)/2) * 1E-3
vr2

45.455

### 3.3: Path 4-4 (Tearout)  (1mm plate)

In [15]:
An = 0.
Ut = 1

In [16]:
Agv = (4*(2*75+35))*1
Agv

740

In [17]:
vr3 = phiu*(Ut*An*pFu + 0.6*Agv*(pFy+pFu)/2) * 1E-3
vr3

123.210

### 3.4: Path 5-3 (Net Section Fracture)  (1mm plate)

![Failure Path](gusset-net-section.svg)

In [18]:
# length 7-3:
l73 = 75 + 35
# length 6-7: (all lines at 45 degrees)
l67 = (75+75)/(2**0.5)
# length 5-6:
l56 = 35*(2**0.5)
# gauge and pitch for line 5-7 (they are equal):
s = g = (l56+l67)/(2**0.5)
# net area:
Ane = An = (l56+l67+l73)*1 - 2*hd*1 + (s*s/(4*g))*1       # 12.3.1

tr1 = phiu*Ane*pFu * 1E-3      # 13.2 iii)
show('l56,l67,l73,s, Ane,tr1')

l56 = 49.5  
l67 = 106.1 
l73 = 110   
s   = 110   
Ane = 245.1 
tr1 = 80.87 


### 3.5: Bearing resistance  (1mm plate)

In [19]:
br1 = 3*phibr*6*1*bd*pFu * 1E-3
br1

120.701

In [20]:
# governing resistance is (the minimum of the above)
r = min(vr1,vr2,vr3,tr1,br1)
r

45.455

### 3.6: Required Thickness

In [21]:
# thickness required, mm
Tf/r

17.600

The governing resistance is block shear (Path 1-3).  The required thickness is thus 17.6mm, so try a 20mm gusset plate.

### 3.7: Try 20mm Gusset Plate

## 4: 20mm Insert Plate (Slotted into HSS)

In [22]:
Dsg = 'HS127x127x6.4'
hss = sst.section(Dsg)
hssT,hssAg = hss.props('T,A')
show('hssT,hssAg')

hssT  = 6.35 
hssAg = 2960 


In [23]:
pW = 35 + 75 + 35   # width of plate based on bolt spacing and edge distance
pT = 20             # thickness of plate based on gusset thickness

### 4.1: Gross Area Yield

In [24]:
Ag = hssAg
Tr = phi*Ag*pFy * 1E-3
show('Tr, Tr >= Tf')

Tr       = 799.2 
Tr >= Tf = False 


Thats close enough.  OK

### 4.2: Net Section Fracture

In [25]:
Ane = pW*pT - 2*hd*pT
Tr = phiu*Ane*pFu * 1E-3
show('Tr, Tr >= Tf')

Tr       = 640.2 
Tr >= Tf = False 


No good - make the plate wider from:

$$
\phi_u(w t - 2\times24t)F_u \ge T_f
$$

$$
w t \ge \frac{T_f}{\phi_u F_u} + 48 t
$$

$$
w \ge \frac{T_f}{\phi_u t F_u} + 48
$$

In [26]:
Tf/(phiu * pT * pFu*1E-3) + 48

169.212

This leads to $(170-75)/2 \approx 50$mm edge distances.  As the plate has to be so wide anyway (I'm relunctant to
make it thicker because of the extra cross-section removed from the HSS), perhaps we should try the 3x2 bolt arrangement and a slightly wider plate.

### 4.3: Try 20mm x 220mm Insert Plate

![Dimensions](insert-3x2-dimensions.svg)

In [27]:
pW = 220
pT = 20

![Paths](insert-3x2-paths.svg)

## 5: Insert Plate - Bolted End

##### Gross Area Yield:  OK, from above  (220 > 145)

##### Net Section Fracture: OK, from above (220 > 168.2)

### 5.1: Block Shear: Path 3-2

In [28]:
An = (75+75+35 - 24*2.5)*pT
Ut = 0.6
Agv = (35+75)*pT

Tr = phiu*(Ut*An*pFu + 0.6*Agv*(pFy+pFu)/2) * 1E-3
show('An, Ut, Agv, Tr, Tr >= Tf')

An       = 2500  
Ut       = 0.6   
Agv      = 2200  
Tr       = 861.3 
Tr >= Tf = True  


### 5.2: Block Shear: Path 3-4

In [29]:
An = (75+75 - 24*2)*pT
Ut = 1.0
Agv = 2*(35+75)*pT

Tr = phiu*(Ut*An*pFu + 0.6*Agv*(pFy+pFu)/2) * 1E-3
show('An, Ut, Agv, Tr, Tr >= Tf')

An       = 2040 
Ut       = 1    
Agv      = 4400 
Tr       = 1406 
Tr >= Tf = True 


### 5.3: Tearout

In [30]:
An = 0
Ut = 1.0
Agv = 6*(35+75)*pT

Tr = phiu*(Ut*An*pFu + 0.6*Agv*(pFy+pFu)/2) * 1E-3
show('An, Ut, Agv, Tr, Tr >= Tf')

An       = 0     
Ut       = 1     
Agv      = 13200 
Tr       = 2198  
Tr >= Tf = True  


### 5.4: Bearing

OK, from above. (see 3.5)

## 6: Insert - Welded End

### 6.1: Size and Length of Weld

Handbook, page 6-186, shows that for $12 \le t \le 20$, minimum fillet size is 6mm.

As the weld of the Insert to the HSS is not near an edge, there is no essnetial maximum size.

### 6.2: Try 6mm fillet weld

In [31]:
# Required length is: (all weld parallel to axis of load)
Tf/(0.67*phiw*(0.707*6)*Xu*1E-3)  # S16 13.13.2.2

857.381

In [32]:
857/4

214.250

Try 220mm of weld in 4 locations, insert plate to HSS.

### 6.3: Net Section Fracture

In [33]:
# central portion is An2
w = 127.
L = 220.
pT = 20.
2*w >= L >= w

True

In [34]:
An2 = .5*w*pT + .25*L*pT
show('w,L,An2')

w   = 127  
L   = 220  
An2 = 2370 


In [35]:
# edge portions are An3's:
w = (220-127)/2.
xbar = w/2.
L >= w

True

In [36]:
An3 = (1-xbar/L)*w*pT
show('w,xbar,An3')

w    = 46.5  
xbar = 23.25 
An3  = 831.7 


In [37]:
Ane = An3 + An2 + An3
Tr = phiu * Ane * pFu * 1E-3
show('Ane,Ane/(pW*pT),Tr,Tr>=Tf')

Ane         = 4033   
Ane/(pW*pT) = 0.9167 
Tr          = 1331   
Tr>=Tf      = True   


**Therefore, 20mm x 220mm Insert Plate OK**

------------------
------------------

## 7: HSS 127x127x6.4

![x-sect](hss-xsect.svg)

In [38]:
# See page 7-88 CISC HB
d = 127
b = (127/2) - (20/2.)
w = t = 6.35
A = d*w + 2*(b-w)*t
xbar = b - (1/(2.*A))*((d-2*t)*w**2 + 2*t*b**2)
Lw = 220
show('d,b,t,A,xbar,Lw, xbar/Lw >= 0.1')

d              = 127   
b              = 53.5  
t              = 6.35  
A              = 1405  
xbar           = 38.93 
Lw             = 220   
xbar/Lw >= 0.1 = True  


In [39]:
An = 2960 - 2*20*6.35      # 20mm slot cut from both sides of HSS
Ane = max((1.1 - xbar/Lw)*An, 0.8*An)
Tr = phiu*Ane*hFu * 1E-3
show('An,Ane,Ane/An,Tr,Tr>=Tf')

An     = 2706   
Ane    = 2498   
Ane/An = 0.9231 
Tr     = 843    
Tr>=Tf = True   


**Therefore, HSS 127x127x6.4 OK**

---------------
-----------------

## 8: Lap Plates - 2 @ 10mm x 220mm

OK, as thickness of 2 plates = thickness of insert plate, and widths are the same.

**Therefore, 2-10mmx220mm Lap Plates OK**

----------------
----------------

## 9: Gusset Plate

Gusset plate OK as thickness and width same as insert plate.

**Therefore, 20mm gusset plate OK**

## 10: Summary

![Summary](design-summary.svg)